In [ ]:
# 🧩 1. Load Required Libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

# 🧠 Why?
# •	pickle: to load .pkl files (like encoders and scalers).
# •	pandas: to handle input data.
# •	load_model: to load the trained .h5 model.

# 📂 2. Load the Trained Model
model = load_model('model.h5')
# 🧠 Why?
# •	We trained the model earlier and saved it as .h5. Now we're loading it for prediction.

model.summary()

# Model: "sequential_7"
# ┌─────────────────────────────┬──────────────────────┬─────────────┐
# │ Layer (type)                │ Output Shape         │ Param #     │
# ├─────────────────────────────┼──────────────────────┼─────────────┤
# │ dense_15 (Dense)            │ (None, 64)           │        832  │
# │ dense_16 (Dense)            │ (None, 32)           │      2,080  │
# │ dense_17 (Dense)            │ (None, 1)            │         33  │
# └─────────────────────────────┴──────────────────────┴─────────────┘
#  Total params: 2,947 (11.52 KB)
#  Optimizer params: 2 (12.00 B)

# 🧠 3. Load the Saved Encoders & Scalers 
with open("one_hot_encoder_geo.pkl", "rb") as f:
    one_hot_encoder_geo = pickle.load(f)

with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)

with open("standard_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# 📌 We saved preprocessing objects (encoders, scaler) to .pkl so we can use exactly the same logic later.

# 📝 4. Input New Customer Data for Prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

# 🔄 5. Convert to DataFrame
input_df = pd.DataFrame([input_data])
# input_df
# ┌─────┬─────────────┬───────────┬────────┬─────┬────────┬───────────────┬───────────────┬────────────┬────────────────┬───────────────────┐
# │     │ CreditScore │ Geography │ Gender │ Age │ Tenure │    Balance    │ NumOfProducts │ HasCrCard  │ IsActiveMember │ EstimatedSalary   │
# ├─────┼─────────────┼───────────┼────────┼─────┼────────┼───────────────┼───────────────┼────────────┼────────────────┼───────────────────┤
# │  0  │     600     │  France   │  Male  │ 40  │   3    │    60000      │       2       │      1     │        1       │       50000       │
# └─

# 🏷️ 6. One Hot Encode 'Geography' Feature
geo_encoded = one_hot_encoder_geo.transform([[input_df["Geography"][0]]])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df
# 📌 Geography is a categorical column (France, Germany, Spain) — we convert it into columns like:
# ┌─────┬────────────────────┬───────────────────────┬────────────────────┐
# │     │ Geography_France   │ Geography_Germany     │ Geography_Spain    │
# ├─────┼────────────────────┼───────────────────────┼────────────────────┤
# │  0  │        1.0         │         0.0           │        0.0         │
# └─────┴────────────────────┴───────────────────────┴────────────────────┘


# 🔄 7. Label Encode 'Gender' Feature 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df
# ┌─────┬─────────────┬───────────┬────────┬─────┬────────┬────────┬────────────────┬────────────┬────────────────┬───────────────────┐
# │     │ CreditScore │ Geography │ Gender │ Age │ Tenure │ Balance│ NumOfProducts │ HasCrCard  │ IsActiveMember │ EstimatedSalary   │
# ├─────┼─────────────┼───────────┼────────┼─────┼────────┼────────┼────────────────┼────────────┼────────────────┼───────────────────┤
# │  0  │     600     │  France   │   1    │ 40  │   3    │ 60000  │       2        │     1      │        1       │       50000       │
# └─────┴─────────────┴───────────┴────────┴─────┴────────┴────────┴────────────────┴────────────┴────────────────┴───────────────────┘

# 🧩 8. Merge All Transformed Features into One DataFrame
input_df = input_df.drop('Geography', axis=1)
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df
# 🧠 Geography is now encoded — so we remove original 'Geography' column and add the one-hot version
# ┌─────┬─────────────┬────────┬─────┬────────┬────────┬───────────────┬────────────┬────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┐
# │     │ CreditScore │ Gender │ Age │ Tenure │ Balance│ NumOfProducts │ HasCrCard  │ IsActiveMember │ EstimatedSalary   │ Geography_France   │ Geography_Germany  │ Geography_Spain   │
# ├─────┼─────────────┼────────┼─────┼────────┼────────┼───────────────┼────────────┼────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┤
# │  0  │     600     │   1    │ 40  │   3    │ 60000  │       2       │     1      │        1       │       50000       │        1.0        │        0.0        │        0.0        │
# └─────┴─────────────┴────────┴─────┴────────┴────────┴───────────────┴────────────┴────────────────┴───────────────────┴───────────────────┴───────────────────┴───────────────────┘

# 📉 9. Scale All Features
input_scaled = scaler.transform(input_df)
input_scaled
# array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
#          0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
#         -0.57946723, -0.57638802]])
# 🧠 We apply the same standardization (mean=0, std=1) used during training.

# 🔮 10. Predict with the Model
prediction = model.predict(input_scaled)
print("Prediction Probability:", prediction[0][0])
# 🧠 Output is a probability — how likely the customer is to leave the bank.
# [1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 74ms/step
# c:\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
#   warnings.warn(
# Prediction Probability: 0.03495315

# ✅ 11. Final Output – Binary Decision
if prediction[0][0] > 0.5:
    print("Customer is likely to churn.")
else:
    print("Customer is NOT likely to churn.")

# Customer is NOT likely to churn.
# 📌 If probability > 0.5 → churn
# 📌 Else → no churn





Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,947 (11.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Prediction Probability: 0.03495315
Customer is NOT likely to churn.


c:\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
